In [52]:
import os
import sys
import scanpy as sc
import pandas as pd

from langchain_openai import ChatOpenAI

sys.path.append(os.path.abspath('..'))
from cellhashtag.utils import *
from cellhashtag._Agent import CellHashtagAgent

# Set up LLMs

In [ ]:

bgi_api_key = 'sk-NYPE5WNGTK781Nox801a5934A9F84a03BfC525359eBaE7B6'
bgi_api_base = 'http://10.224.28.80:3000/v1'
def call_chatllm_openai(api_key, api_base, model_name):
    llm = ChatOpenAI(
        openai_api_key = api_key,
        openai_api_base=api_base,
        model = model_name)
    return llm
chat_llm = call_chatllm_openai(bgi_api_key, bgi_api_base, 'deepseek-v3-hs')
think_llm = call_chatllm_openai(bgi_api_key, bgi_api_base, 'deepseek-v3-hs')

tavily_api = "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"

# Perpare test data

In [ ]:
adata = sc.read_h5ad('../cellhashtag/data/example.h5ad')
cell_markers_df = pd.read_csv('../cellhashtag/data/example_cellmarkers.csv',index_col=0)
print('Check cell markers dataframe:')
cell_markers_df

In [ ]:
agent = CellHashtagAgent(web_scraper_api_key = None)

In [ ]:
draw_graph(agent.graph)

In [ ]:
draw_graph(agent.subgraph)

In [ ]:
output_adata, output_states = agent.run(
                                        llm=think_llm, 
                                        adata= adata,
                                        cell_marker_df_dir = '../cellhashtag/data/example_cellmarkers.csv',
                                        )

In [ ]:
def rename_cluster(dataframe,)

In [ ]:
res_df = pd.read_csv("AnnoResDF.csv",index_col=0)
res_df['cluster'] = res_df['cluster'].astype(str)
res_df

In [ ]:
temp_df

In [ ]:
temp_df = res_df.copy().set_index('cluster')
anno_dic_raw = temp_df['cell type'].to_dict()
anno_dic= temp_df['norm. cell type'].to_dict()

In [ ]:
anno_dic_raw

In [ ]:
adata.obs["Cell#"] = adata.obs["leiden"]
adata.obs["Cell#"] = adata.obs["Cell#"].map(anno_dic)
adata.obs["Cell#_raw"] = adata.obs["leiden"]
adata.obs["Cell#_raw"] = adata.obs["Cell#_raw"].map(anno_dic_raw)

In [ ]:
'umap' not in adata.uns

In [ ]:
sc.pl.umap(adata,color = ['Cell#','Cell#_raw','leiden'],ncols=1)

In [ ]:
res_df['cluster'] = res_df['cluster'].astype(str)
res_df.set_index('cluster',inplace=True)
ct_dic = res_df['cell type'].to_dict()

In [ ]:
adata.obs['Cell# raw'] = adata.obs['leiden']

adata.obs['Cell# raw'].map(ct_dic)

In [ ]:
ct_dic

In [ ]:
subgraph = init_subgraph()
#draw_graph(subgraph)

In [ ]:
all_markers = []
for string in cell_markers_df['cell_markers'].tolist():
    exec("all_markers+="+string)
hvgs = get_cluster_hvgs(adata)
gene_list = list(set(all_markers+hvgs))

mask = adata.obs['leiden'] == '1'
exp_summary  = get_exp_summary(adata[mask], gene_list)

cluster = '1'
max_iter = 5
exp_summary_dict ={'1': exp_summary}
metadata_dict = {'1': 'human with colorectal cancer, sample from normal colorectal mucosa '}
cell_markers_table = df2markdownTable(cell_markers_df)
anno_strategy = 'Primary'

In [ ]:
config = {"configurable": {"thread_id": "1"}}
subgraph_res = subgraph.invoke(
    input = {
        #pars
        "anno_llm": think_llm,
        "max_iter" : 5,
        #input
        "cluster" : cluster,
        "anno_strategy" : "secondary",
        "cell_markers_table" : cell_markers_table,
        "exp_summary_dict": exp_summary_dict,
        "metadata_dict": metadata_dict,
        "anno_critique" :"",
        "n_iter": 0
    },
    config = config,
)

In [ ]:
subgraph_res['anno_result']['functional group'] = str(subgraph_res['anno_result']['functional group'] )

In [ ]:
test = [pd.DataFrame([subgraph_res['anno_result']]), pd.DataFrame([subgraph_res['anno_result']])]

In [ ]:
test = pd.read_csv("AnnoResDF.csv")

In [ ]:
adata.obs['Cell#'] = adata.obs['leiden']
adata.obs['Cell#'] = adata.obs['Cell#'].replace(test)

In [ ]:
sc.pl.umap(adata, color='Cell#')

In [ ]:
test

In [ ]:
test = pd.read_csv("AnnoResDF.csv")
test['cluster'] = test['cluster'].astype('str')
test = test[['cluster','cell type']]
test = test.set_index('cluster').to_dict()['cell type']

In [ ]:
test

# Packaging

In [ ]:
#
#Packaging annotator
#
import anndata as ad
import pandas as pd

# The annotator is combinatied from a parent graph (for collect importmation and parse the result) and a subgraph (for annotation using those infromation).




    
#Define state for subgraph
class SubGraphState(TypedDict):
    """
    A dictionary structure to hold the state of subgraph operations including configuration, input, internal processing, 
    and output fields for cell type annotation tasks.
    """

    # Configuration Parameters
    anno_llm: ChatOpenAI  # LLM (Language Model) configured for annotations. Used to generate or assist in creating annotations.
    max_iter: int  # Maximum number of iterations allowed for annotation processes.

    # Input Data
    cluster: str  # Name of the cluster being processed.
    anno_strategy: str  # Strategy used for annotation, options are 'primary' or 'secondary'.
    cell_markers_table: str  # Information on cell types and their markers formatted as a markdown table.
    exp_summary_dict: dict  # Expression summary of marker genes and highly variable genes (HVGs) for each cluster. Keys are cluster names, values are dictionaries containing expression tables for each gene.
    metadata_dict: dict  # Useful metadata for annotation, formatted as a markdown table.

    # Internal State During Processing
    n_iter: int  # Current iteration number during the annotation process.
    anno_response: str  # Response generated by the LLM for annotation purposes.
    anno_celltype: str  # Determined cell type for each cluster after annotation.
    anno_critique: str  # Critiques or feedback on the annotation process or results.
    decision: str  # Decision made based on critiques, guiding further actions or finalizing annotations.

    # Output Data
    anno_result: dict  # Final annotation result for the given cluster, structured as required for downstream analysis or reporting.

#define subgraph
#================================
#def nodes
#--------------------------------
def node_annotation(
        state: SubGraphState
        ):
    #pass parameters
    llm = state["anno_llm"]
    cluster = state["cluster"]
    strategy = state["anno_strategy"]
    exp_summary_dict = state["exp_summary_dict"]
    metadata_dict = state["metadata_dict"]
    cell_markers_table = state["cell_markers_table"]
    critique = state["anno_critique"]
    n_iter = state["n_iter"]
    #convert to markdown table
    exp_summary_table = exp_summary_dict[cluster]
    metadata_table = metadata_dict[cluster]
    #convert critique
    if len(critique) > 1:
        critique = "\nYou and your coworkers are collaborating on multiple rounds of annotations and corrections, and your coworkers will be evaluating your previous annotations, if not the first round; consider their evaluations before you annotate:\n" + critique
    #set up instructions for different strategies
    if strategy.lower() == 'secondary':
        instruction = "Since the current annotation is aimed at subpopulations of specific cell types that are very case specific, you do not necessarily have to adhere to the given cell types and their markers. But please still respect the given hierarchy of reference cell types as much as possible."
    else:
        instruction = "Since the current annotation is aimed at the whole population of cell types, you should adhere to the given cell types and their markers as much as possible."
#prompting
    annotation_prompt = """
You are a bioinformatics expert in the field of single-cell RNAseq.
You are going to annotate the cell type of the following cluster, its expression status is:
{exp_summary_table}

The cluster is from the following sample:
{metadata_table}

You can using the following cell types and cell markers earlier provided by another expert for hint:
{cell_markers_table}

{instruction}
{critique}

In your answer, please first analyze the gene expression of this cluster and then annotate this cluster with your knowledge and the provided cell types and cell markers and give reasons, the reasons should include:
    1. key genes of the target cell type; 
    2. positively expressed genes that support the annotation results; 
    3. negatively expressed genes that support the annotation results; 
    4. conflicting or potentially confusing genes; and 
    5. your overall conclusions. 

Note that You don't have to follow the referenced markers for annotations, and you can take a certain amount of liberties when you think there is a more appropriate naming convention for the current subcluster. But be sure to give the cluster an annotation and a good reason for it.

Finally, transcribe the results of the annotation to the end of your answer, using following json array format:
```json
[
    "annotated cell type", 
]
```
"""
    #build chain
    prompt = PromptTemplate(
        input_variables=[
            "exp_summary_table", 
            "metadata_table", 
            "cell_markers_table", 
            "instruction", 
            "critique"],
        template=annotation_prompt
    )
    chain = prompt | llm
    #call llm
    print(f"Annotating cluster {cluster}...")
    response = chain.invoke({
        "exp_summary_table": exp_summary_table,
        "metadata_table": metadata_table,
        "cell_markers_table": cell_markers_table,
        "instruction": instruction,
        "critique": critique
    })
    #parse response
    cell_type = parse_jsonfromcontent(response.content)
    cell_type = cell_type[0]
    #recod interation
    n_iter += 1
    return {
        "anno_response": response.content,
        "anno_celltype": cell_type,
        "n_iter": n_iter,
    }


def node_critic(state: SubGraphState):
    #pass parameters
    llm = state["anno_llm"]
    cluster = state["cluster"]
    cell_type = state["anno_celltype"]
    draft_response_wreason = state["anno_response"]
    exp_summary_dict = state["exp_summary_dict"]
    metadata_dict = state["metadata_dict"]
    cell_markers_table = state["cell_markers_table"]
    #convert to markdown table 
    exp_summary_table = exp_summary_dict[cluster]
    metadata_table = metadata_dict[cluster]
    #prompting
    annotation_critic_prompt = """
You are a bioinformatics expert in the field of single-cell RNAseq.
One of your coworkers has annotated this cluster as {cell_type} and given the results, you need to help him check the results and give your comments and reasons.
The expression of this cluster is as follows:
{exp_summary_table}

The cluster is from the following sample:
{metadata_table}

The reason for your colleague's annotation is as follows:
{draft_response_wreason}

The list of referenced markers is as follows:
{cell_markers_table}

Give your evaluation and reasons for your colleague's annotation results. Then, append final decision to the end of your answer in the following json array format:
```json
{{
    "final decision": str //  Chose from `Approved` if you think the given annotation result of is reasonable, otherwise add `Disapproved`
}}
"""
    #build chain
    prompt = PromptTemplate(
        input_variables=[
            "cell_type", 
            "exp_summary_table", 
            "metadata_table", 
            "draft_response_wreason", 
            "cell_markers_table"],
        template=annotation_critic_prompt
    )
    chain = prompt | llm 
    #call llm
    print(f"Checking annotation for cluster {cluster}...")
    response = chain.invoke({
        "cell_type": cell_type,
        "exp_summary_table": exp_summary_table,
        "metadata_table": metadata_table,
        "draft_response_wreason": draft_response_wreason,
        "cell_markers_table": cell_markers_table
    })
    #parse response
    decision_dic = parse_jsonfromcontent(response.content)
    decision = decision_dic["final decision"]
    return {
        "anno_critique": response.content,
        "critic_decision": decision,
    }

def node_finalize(state: SubGraphState):

    cluster = state["cluster"]
    cell_type = state["anno_celltype"]
    anno_response = state["anno_response"]
    functional_anno = "not applied"
    finnal_result = {
        "cluster": cluster,
        "cell type": cell_type,
        "functional group": functional_anno,
        "reason": anno_response,
    }
    print("Final annotation of",cluster,"is",cell_type)
    return {
        "anno_result": finnal_result,
    }

def node_funcional_characterization(state: SubGraphState):
    #pass parameters
    llm = state["anno_llm"]
    cluster = state["cluster"]
    exp_summary_dict = state["exp_summary_dict"]
    metadata_dict = state["metadata_dict"]
    cell_type = state["anno_celltype"]
    anno_result = state["anno_result"]
    #convert to markdown table
    exp_summary_table = exp_summary_dict[cluster]
    metadata_table = metadata_dict[cluster]
    #prompting
    functional_prompt = """
You are a bioinformatics expert specializing in functional characterization of single-cell RNAseq subpopulations.
You are tasked with performing functional profiling of the following subcluster, which has been preliminarily annotated as {cell_type}.

Its detailed expression profile is:
{exp_summary_table}

The metadata of the current cluster are as follows, and inferences about functions and features can be made based on this meta information:
{metadata_table}

In your analysis, you should:
1. Identify hallmark biological processes (e.g., stemness, apoptosis, cytokine production) 
2. Detect cellular state indicators (e.g., proliferative, quiescent, activated)
3. Recognize specialized functional programs (e.g., metabolic activity, stress response)

Your annotation rationale must explicitly address:
1. Core functional markers driving the characterization
2. Supportive co-expressed genes reinforcing the functional state
3. Exclusion markers ruling out alternative functional states
4. Ambiguous/conflicting expression patterns requiring interpretation
5. Integration of evidence for final functional designation

Annotation guidelines:
- Combine known conventions with novel descriptors when justified (e.g., "Senescent_epithelial", "IFNγ-high_CD8+_T")
- Consider gradient states using modifiers like "progenitor", "transitioning", or "activated"
- Maintain biological plausibility while allowing subpopulation-specific terminology

Present your analysis in this structure:
1. **Functional Process Identification**
- Highlight key pathways (e.g., "Wnt signaling activation")
- Note cell cycle status indicators (e.g., G2/M phase genes)

2. **State Characterization**
- Identify stress/differentiation signatures
- Detect secretory/effector programs

3. **Consistency Assessment**
- Verify alignment with parent cell type biology
- Resolve expression conflicts

Conclude with functional annotation append to your reason(note: you don't need to label the cell type), in this JSON format:
```json
[
    "functional_annotation", 
]
"""
    #build chain
    prompt = PromptTemplate(
        input_variables=[
            "cell_type", 
            "exp_summary_table", 
            "metadata_table"],
        template=functional_prompt
    )
    chain = prompt | llm | JsonOutputParser()
    #call llm
    print(f"Functional characterization for cluster {cluster}...")
    response = chain.invoke({
        "cell_type": cell_type,
        "exp_summary_table": exp_summary_table,
        "metadata_table": metadata_table,
    })
    #parse response
    functional_anno = response
    anno_result["functional group"] = functional_anno
    return {
        "anno_result": anno_result,
    }


#def decision functions
#--------------------------------
def decfun_decision(state: SubGraphState):
    n_iter = state["n_iter"]
    cluster = state["cluster"]
    max_iter = state["max_iter"]
    decision = state["critic_decision"]
    if n_iter >= max_iter:
        print(f"Maximum number of annotation iterations reached for cluster {cluster}.")
        return "finalize"
    else:
        if decision.lower() == "approved":
            print(f"Annotation result for cluster {cluster} is satisfied.")
            return "finalize"
        else:
            print(f"Annotation result for cluster {cluster} is not yed satisfied, for round",n_iter)
            return "reflex"

def decfun_strategy(state: SubGraphState):
    strategy = state["anno_strategy"]
    if strategy.lower() == "secondary":
        return "functional"
    else:
        return "end"

#Build subgraph 
#--------------------------------
subgraph_builder = StateGraph(SubGraphState)
#nodes
subgraph_builder.add_node("annotation", node_annotation)
subgraph_builder.add_node("critic", node_critic)
subgraph_builder.add_node("finalize", node_finalize)
subgraph_builder.add_node("functional", node_funcional_characterization)
#edges
subgraph_builder.add_edge(START, "annotation")
subgraph_builder.add_edge("annotation", "critic")
subgraph_builder.add_conditional_edges(
    "critic", 
    decfun_decision,
    {
        "finalize":"finalize",
        "reflex":"annotation"
    }
    )
subgraph_builder.add_conditional_edges(
    "finalize",
    decfun_strategy,
    {
        "functional":"functional",
        "end":END
    }
    )
subgraph_builder.add_edge("functional", END)
#compile
subgraph = subgraph_builder.compile(checkpointer=MemorySaver())

In [ ]:
from langgraph.types import Send
#map-reduce with subraph test
class mainstate(TypedDict):
    A: str #pass not change
    B: str
    C: list #list of Es
    D: Annotated[list, operator.add]

class mapstate(TypedDict):
    A: str #pass not change
    B: str
    c: str #list of Es


class substate(TypedDict):
    A: str 
    E: str #element of C


def subnode_1(state: substate) -> substate:
    print(state["A"])
    D = state["E"] + "_reduced_from_sub"
    return {
        "D":[D]
    }

subbuilder = StateGraph(substate)
subbuilder.add_node("subnode_1",subnode_1)
subbuilder.add_edge(START, "subnode_1")
subbuilder.add_edge("subnode_1",END)

subgraph = subbuilder.compile()

def mainnode_1(state: mainstate) -> mainstate:
    print(
        "A",state["A"],
        "\nB",state["B"],
        "\nC",state["C"],
        )
    
    return {}

def call_subgraph(state: mapstate) -> mainstate:
    subgraph_input = {
        "A": state["A"],
        "E": state["c"],
    }
    res = subgraph.invoke(subgraph_input)
    return {"D":[res]}

def mainnode_2(state: mainstate) -> mainstate:
    print(state["D"])
    return {}


def map_function(state: mainstate):
    return [
        Send("subgraph", {"A":state["A"],"c":c}) for c in state["C"]
    ]

builder = StateGraph(mainstate)
builder.add_node("node1",mainnode_1)
builder.add_node("subgraph",call_subgraph)
builder.add_node("node2",mainnode_2)

builder.add_edge(START, "node1")
builder.add_conditional_edges(
    "node1",
    map_function,
    [
        "subgraph",
    ]
)
builder.add_edge("subgraph", "node2")
builder.add_edge("node2", END)
graph = builder.compile()

In [ ]:
graph.invoke({'A':'A','B':'B','C':['1','2','3']})

In [ ]:
from functools import partial
from concurrent.futures import ThreadPoolExecutor, as_completed
from langgraph.types import Send
#Define state for parent-graph
class MainGraphState(TypedDict):
    #parameters
    max_iter: int #max iteration for annotation
    anno_llm: ChatOpenAI #LLM for annotation, models with reasoning ability is recommended

    #input
    input_adata: ad.AnnData 
    cluster_key: str #clusters for annotation, e.g. 'leiden'
    maker_df_dir: str #local direction of cell marker df
    marker_offered: bool #if marker df offered and successfully load

    #internal
    metadata_summary: str #summary of input anndata
    #metadata_col: list # list of metadata related columns from anndata
    metadata_dict: dict #dictionary for metadata, where key is each cluster name and metadata for each cluster as item

    target_cell_types: list #target cell types to annotate
    anno_strategy: str #strategy for annotation, 'primary' or 'secondary' 
    cell_markers_table: str #markdown table for cell types and their markers
    clusters: list #list of clusters in cluster_key, use for map
    exp_summary_dic: dict #expression summary of marker genes and hvgs for each cluster, where cluster name as key and expression table for each genes as item
    anno_results: Annotated[list[dict], operator.add] #annotation result for each cluster from subgraph
    #output
    anno_result_df: pd.DataFrame #annotation result for each cluster, where [cluster name], [annotated cell type], [reason], [functional group] as columns
    output_adata: ad.AnnData #output anndata with cell type annotation


class MapGraphState(TypedDict):
    #parameters
    anno_llm: ChatOpenAI #pass not chage
    max_iter: int #pass not chage
    #input
    cluster: str #mapped from clusters from MainGraphState
    anno_strategy: str #pass not chage
    cell_markers_table: str #pass not chage
    exp_summary_dic: dict #pass not chage
    metadata_dict: dict #pass not chage

def node_get_metadata(state:MainGraphState):
    """
    Extract useful metadata columns from an AnnData object for cell type annotation using a language model (LLM).
    
    This function:
    1. Generates a summary of the AnnData object.
    2. Uses an LLM to select metadata columns relevant to cell type annotation.
    3. Summarizes the selected metadata for the entire dataset and per cluster.
    
    Parameters:
    - adata (AnnData): AnnData object containing single-cell RNA sequencing data.
    - llm: Language model instance used to select metadata columns.
    - group (str): Column name in adata.obs for clustering (default: 'leiden').
    
    Returns:
    - useful_meta_cols (list): List of metadata column names selected by the LLM.
    - metadata_summary (str): Summary of selected metadata columns for the entire dataset.
    - metadata_dict (dict): Summaries of selected metadata columns for each cluster.
    
    """
    #--------------------------------
    #pass parameters
    llm = state["anno_llm"]
    group = state["cluster_key"]
    adata = state["input_adata"]

    #--------------------------------
    #data presception, step1, get over all presception
    print("Getting summary of given data...")
    data_type, adata_summary = data_perception(adata)
    print("given data is",data_type)
    #--------------------------------
    #call for llm
    chose_column_prompt = """
    You are a biologist who is familiar with the metadata of the dataset.
    You are given a anndata with the following summary:
    {adata_summary}
    Please choose the metadata column that you think is most useful for cell type annotatation in adata.obs, for example we need to know the sample species, tissue and diease for correct annotation. And if there is pre-annotated cell type(s), also return the column name for future use.

    Please return all columns that are useful for cell type annotation in following json array format:
    ```json
    [
        "column1", 
        "column2", 
        ...
    ]
    ```
    """
    prompt = PromptTemplate(
        input_variables = ["adata_summary"],
        template = chose_column_prompt,
    )
    chain =  prompt | llm | JsonOutputParser()
    i = 0 
    while i<3:
        try:
            meta_cols =  chain.invoke({"adata_summary": adata_summary})
            break
        except:
            i+=1

    #--------------------------------
    #data presception, step2, get detailed presception of each column, for whole data and each cluster
    print("Collecting data metadata summary...")
    metadata_summary =  get_dfcol_summaries(adata.obs, meta_cols)
    metadata_dict = {}
    for cluster in adata.obs[group].unique():
        metadata_dict[cluster] = get_dfcol_summaries(adata[adata.obs[group] == cluster].obs, meta_cols)

    

    return {
        "metadata_summary": metadata_summary,
        "metadata_dict": metadata_dict,
    }

def node_get_target_celltypes(state:MainGraphState):
    #--------------------------------
    #pass parameters
    llm = state["anno_llm"]
    metadata_summary = state["metadata_summary"]
    df_dir = state["maker_df_dir"]
    #--------------------------------
    #call for llm
    chose_celltype_prompt = """
    As a biologist familiar with the annotation of single-cell RNA-seq data, you will handle an anndata object containing specific metadata. 
    Your task is to select the most suitable target cells for cell type annotation based on the provided sample metadata, considering factors such as disease status, sample tissue origin, and species information.

    The metadata summary from given anndata.obs:
    {metadata_summary}

    ### Task Steps

    1. **Understand Sample Background**:
    - Analyze the provided `metadata` to understand whether the samples involve healthy controls versus disease states, if there are any drug interventions, etc. These factors are crucial for formulating an annotation strategy.

    2. **Determine Annotation Strategy**:
    - **Primary Cell Type Annotation**: Based on the sample type (e.g., PBMC samples typically include immune cells), choose the most relevant cell types as primary annotation targets. This strategy is generally used for samples with higher specificity, such as human and animal organ samples.
    - **Secondary Cell Type or Functional Annotation**: On top of the primary cell types, consider further subdividing cell types (e.g., distinguishing CD4+ T cells from CD8+ T cells within T cells). However, avoid having both parent and child cell types at the same resolution level to prevent classification confusion. This strategy is generally used for samples with relatively low specificity, such as cell subtypes of major cell types that are already annotated, cell lines cultured in vitro, and so on.

    3. **Select Appropriate Annotation Targets**:
    - Based on the purpose of biological differential analysis, precisely identify the cell types that need annotation to support subsequent marker identification. Ensure that the names of the annotated cell types are common and easily recognizable.

    ### Important Notes
    - In your response, first list the primary cell types suitable for the current dataset as annotation targets.
    - Then, re-normalize these targets according to the hierarchical levels of cell types. For example, if annotations are already made at the level of T cells and B cells, try not to subdivide into CD4+ T cells and CD8+ T cells; Th2 and Treg should never appear at the same resolution level as T cells and B cells. Ensure that cell types and their subsets do not appear together in the annotation results to avoid ambiguous categorization.

    By following this approach, you can effectively provide precise cell type annotation recommendations for single-cell RNA-seq data analysis.

    Please return as many as possible cell types that are suitable targets for annotations, in the following json format:
    ```json
    {{
        "annotation strategy": str // Chose from `primary` which indicate major cell type annotations, or `secondary` for cell subpopulation and/or functional group annotations.
        "cell_types": list // major cell type or cell subpopulation names, e.g. ["cell type 1", "cell type 2", ...].
    }}
    
    ```
    """
    prompt = PromptTemplate(
        input_variables = ["metadata_summary"],
        template = chose_celltype_prompt,
    )
    chain =  prompt | llm | JsonOutputParser()
    i = 0
    print("Looking for target cell types for annotation...")
    while i < 3:
        try:
            response =  chain.invoke({"metadata_summary": metadata_summary})
            #--------------------------------
            #parse parameters
            strategy = response["annotation strategy"]
            cell_types = response["cell_types"]
            break
        except:
            i+=1
    cell_types_str = "Recommended annotation target cell types are:\n"
    for cell in cell_types:
        cell_types_str += cell + '\n'
    print(cell_types_str)

    #--------------------------------
    #trying to use provided cell markers
    try:
        marker_df = pd.read_csv(maker_df_dir)
        print("Cell marker dataframe loaded.")
        try:
            cell_markers_table = df2markdownTable(marker_df)
            marker_offered = True
        except:
            print("Failed to convert cell marker dataframe to markdown.")
            marker_offered = False
            cell_markers_table = ""
    except:
        marker_offered = False
        cell_markers_table = ""

    return {
        "anno_strategy": strategy,
        "target_cell_types": cell_types,
        "marker_offered":marker_offered,
        "cell_markers_table": cell_markers_table,
    }

def scrape_markers(
        cell_type, 
        llm, 
        query,
        ):
    """
    Scrape and retrieve marker genes for a given cell type using web search results and a language model (LLM).

    This function:
    1. Performs a web search based on the cell type and query.
    2. Formats the search results into a readable string.
    3. Uses a language model to recall and refine marker genes based on the search results.
    4. Returns the LLM's full response and the extracted list of markers.

    Parameters:
    - llm: Language model instance used to process and refine the marker list.
    - cell_type (str): The cell type for which marker genes are being retrieved (e.g., "T-cell").
    - query (str): The search query to retrieve relevant web content (e.g., "marker genes").

    Returns:
    - response_content (str): The complete response from the LLM, including its thought process.
    - markers (list): A list of marker genes extracted from the LLM's response in JSON format.
    """
    #web scraping 
    print(f"Trying to fetch cell markers for {cell_type} from web.")
    try:
        web_res = web_retriever.invoke(cell_type + ' ' + query)
        #parse search result
        search_res = '============\n'
        for i, doc in enumerate(web_res, start=1):
            search_res += f"Web page {i}:\n{doc.page_content}\n\n"
    except:
        print("Web search failed")
        search_res = "Web search failed, please use your own memory and knowledge about cell types and thire markers."
    #process search result
    get_markers = """
    You are a bioinformatics expert in the field of single-cell RNAseq.
    Your task is to provide researchers with marker genes for {cell_type}. Follow these steps:
    1. Recall the key marker genes for this cell type based on your knowledge.
    2. Refine the list by adding or removing markers based on the web search results, providing reasons for each change.
    3. Output your thought process and the final list of markers in the specified JSON format.

    Web search results:
    {search_results}
    
    Please output your thought process along with an organized cell types markers that are suitable targets for scRNAseq annotation, the markers in the following json array after your thought process:
    ```json
    [
        "marker1", 
        "marker2", 
        ...
    ]
    ```
    """
    #build chain
    prompt = PromptTemplate(
        input_variables = ["cell_types", "search_results"],
        template = get_markers,
    )
    chain = prompt | llm 
    
    #invoke llm
    response =  chain.invoke(
        {
            "cell_type": cell_type,
            "search_results": search_res,
        })
    #convert to python list
    markers = parse_jsonfromcontent(response.content)
    
    return response.content, markers

def node_scrape_markers(state: MainGraphState):
    """
    Scrapes cell type markers based on provided metadata and cell types.
    
    This function uses a language model to generate a search query prefix from the metadata summary.
    It then scrapes markers for each target cell type using this query in parallel.
    The results are saved to a CSV file and returned as a markdown table.
    
    Parameters:
    state (MainGraphStat): A state object containing necessary parameters.
        - anno_llm: The language model for annotation.
        - metadata_summary: Summary of metadata from collected data.
        - target_cell_types: List of cell types to scrape markers for.
    
    Returns:
    dict: A dictionary containing the markdown table of cell types and their markers.
    """
    #--------------------------------
    #parse parameters
    llm = state["anno_llm"]
    metadata_summary = state["metadata_summary"]
    cell_types = state["target_cell_types"]
    print("Scraping cell markers...")

    #--------------------------------
    #generate search query
    search_query_prompt = """
    You are a bioinformatics expert in the field of single-cell RNAseq.
    Based on the metadata of collected data below, generate a <20-word search prefix for cell type markers:
    {metadata_summary}
    
    Note:
    1. You just need to organize the species, sampling tissue, etc. as a prefix and I'll add the specific cell type before your search query, example: "[cell type] markers for human in breast, breast cancer",
    2. Provide only the search query, without any intermediate thought process.
    """
    prompt = PromptTemplate(
        input_variables = ["metadata_summary"],
        template = search_query_prompt,
    )
    query_chain = prompt | llm | StrOutputParser()
    query = query_chain.invoke({"metadata_summary": metadata_summary})

    #--------------------------------
    #scape cell markers in parallel
    process_marker_scrape = partial(
        scrape_markers,
        llm = llm, 
        query = query,
        )
    
    all_dfs = []
    errors = []
    with ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(process_marker_scrape, cell_type):cell_type for cell_type in cell_types}
        for future in as_completed(futures):
            cell_type = futures[future]
            try:
                reason, markers = future.result()
                df = pd.DataFrame([{
                    "cell_type": cell_type,
                    "markers": markers,
                    "reason": reason,
                }])
                all_dfs.append(df)
            except Exception as exc:
                errors.append((cell_type, exc))
                #print(f'{cell_type} generated an exception: {exc}')
    # print errors if there are any
    if errors:
        print("The following cell types had errors during scraping:")
        for cell_type, exc in errors:
            print(f"{cell_type}: {exc}")

    #--------------------------------
    #parse result and save to csv
    cell_markers_df = pd.concat(all_dfs,ignore_index=True)
    cell_markers_df.to_csv("scraped_cell_markers.csv", encoding="utf-8-sig", index=False)
    print("Cell type and marker list saved to csv.")
    cell_markers_table = df2markdownTable(cell_markers_df[['cell_type','markers']])
    return {
        "cell_markers_table":cell_markers_table,
    }

def node_get_expresssummary(state: MainGraphState):
    """
    Generates expression summaries for clusters in a single-cell RNAseq dataset.
    
    This function processes the AnnData object to extract marker genes and highly variable genes (HVGs)
    for each cluster. It then computes expression summaries for these genes in each cluster.
    
    Parameters:
    state (MainGraphStat): A state object containing necessary parameters.
        - input_adata: AnnData object containing the single-cell RNAseq data.
        - cluster_key: The key in adata.obs that defines the cluster assignments.
    
    Returns:
    dict: A dictionary containing:
        - clusters: List of unique cluster identifiers.
        - exp_summary_dict: A dictionary with clusters as keys and their expression summaries as values.
    """
    import ast
    import scanpy as sc
    #--------------------------------
    #parse parameters
    adata = state["input_adata"]
    group = state["cluster_key"]
    clusters = adata.obs[group].unique().tolist()
    print("Getting data gene expressing summary...")

    #--------------------------------
    #get cell markers 
    cell_markers_df = pd.read_csv("scraped_cell_markers.csv")
    marker_genes = []
    for _, row in cell_markers_df.iterrows():
        if type(row['markers']) == list:
            marker_genes += row['markers']
        elif type(row['markers']) == str:
            marker_genes += ast.literal_eval(row['markers'])

    #--------------------------------
    #get hvgs for each cluster
    if 'rank_genes_groups' not in adata.uns:
        sc.tl.rank_genes_groups(adata, groupby=group)
    hvgs = []
    for clst in clusters:
        hvgs += list(adata.uns['rank_genes_groups']['names'][clst][:5])
    gene_list = list(set(marker_genes + hvgs))

    #--------------------------------
    #get expression summary
    exp_summary_dict = {}
    for clst in clusters:
        mask = adata.obs[group] == clst
        exp_sum = get_exp_summary(adata[mask],gene_list)
        exp_summary_dict[clst] = exp_sum

    return {
        "clusters":clusters,
        "exp_summary_dict":exp_summary_dict,
    }
    
def node_annotation_mapped(state: MapGraphState):
    subgraph_input = {
        #pars
        "anno_llm": state["anno_llm"],
        "max_iter" : state["max_iter"],
        #input
        "cluster" : state["cluster"],
        "anno_strategy" : state["anno_strategy"],
        "cell_markers_table" : state["cell_markers_table"],
        "exp_summary_dict": state["exp_summary_dict"],
        "metadata_dict": state["metadata_dict"],
        #sub graph only
        "anno_critique" :"",
        "n_iter":0
    }
    config = {"configurable": {"thread_id": "1"}}
    print("Calling subgraph for annotation...")
    #call subgraph
    subgraph_res = subgraph.invoke(subgraph_input, config=config)

    return {"anno_results":[subgraph_res['anno_result']]}
    


def node_normalize_celltype(state: MainGraphState):
    #pass parameters
    llm = state["anno_llm"]
    metadata_summary = state["metadata_summary"]
    anno_results = state["anno_results"]
    adata = state["input_adata"]
    print("Normalizing major cell type annotations...")
    #convert annotation result to dataframe
    res_df_list = []
    for res_dic in anno_results:
        df_temp = pd.DataFrame([res_dic])
        res_df_list.append(df_temp)
    anno_results_df = pd.concat(res_df_list,ignore_index=True)
    anno_results_df.to_csv('AnnoResDF.csv')
    print('Raw annotation result saved as csv.')
    #get unique cell types
    annotated_cell_types = anno_results_df["cell type"].unique().tolist()
    #call llm
    normaliztion_prompt = """
    As a biology researcher specializing in diverse cell types, you have been entrusted with the task of refining cell type annotations derived from scRNA-seq data processed by a colleague. The current annotations exhibit variability in granularity and require standardization to ensure uniformity across the dataset.

    Your objective is to normalize these annotations by leveraging the provided sample metadata and the comprehensive list of cell types identified. This process involves several key steps:
        1.Purification of Annotations: Begin by eliminating any extraneous or irrelevant information present within the annotation results to streamline the dataset.
        2.Hierarchical Grouping: Consolidate subtypes under their corresponding parent cell types to achieve a coherent hierarchical structure. This step ensures that all annotations are represented at a consistent level of detail. e.g. "CD4+ memory T cells" should be consolidated under "CD4+ T cells" to  "CD8+ T cells" 
        3.Standardization of Nomenclature: Apply appropriate capitalization rules to each cell type designation for clarity and professionalism. Proper formatting not only enhances readability but also aligns with scientific conventions.
    By meticulously executing these steps, you will produce a refined and standardized set of cell type annotations that maintain consistency throughout the entire dataset.

    **Inputs:**
    - Sample Metadata: 
    `{metadata_summary}`

    - Cell Types:
    `{annotated_cell_types}`

    **Output Requirement:**
    Please provide the normalized cell types in JSON format as follows:
    ```json
    {{
        "old_cell_type_name_1": str,\\ "normalized_cell_type_name_1"
        "old_cell_type_name_2": str, \\"normalized_cell_type_name_2",
        ...
    }}
    ```
    Ensure each key-value pair accurately reflects the mapping from the original to the normalized cell type names, facilitating easier comparison and analysis of the scRNA-seq data.
    """
    #build chain
    prompt = PromptTemplate(
        input_variables=["metadata_summary", "annotated_cell_types"],
        template=normaliztion_prompt
    )
    i = 0
    while i < 3:
        try:
            chain = prompt | llm | JsonOutputParser()
            response = chain.invoke({
                "metadata_summary": metadata_summary,
                "annotated_cell_types": annotated_cell_types
            })
            #get back to dataframe
            anno_results_df['norm. cell type'] = anno_results_df["cell type"]
            anno_results_df['norm. cell type'].replace(response, inplace=True)
            break
        except:
            i+=1
    #save to csv 
    anno_results_df.to_csv('AnnoResDF.csv')
    print('Normalized annotation result saved as csv.')
    anno_dic = anno_results_df[['cluster','cell type']].to_dict()
    #get back to adata
    adata.obs["cell#"] = adata.obs["leiden"]
    adata.obs["cell#"] = adata.obs["cell#"].replace(anno_dic)
    return {
        "anno_result_df": anno_results_df,
        "output_adata": adata,
    }


#----------------------------------------------------------------
#Define decision functions for subgraph
def decision_scrapemarkers(state:MainGraphState):
    marker_offered = state["marker_offered"]
    if marker_offered:
        return "scrape markers"
    else:
        return "skip scrape"
    
#----------------------------------------------------------------
#Define map-reduce functions for subgraph
def map_annotation(state:MainGraphState):
    #return a list of `Send` objects, each `Send` object consists of the name of a node in the graph
    return [Send("annotation_subgraph", {
        "anno_llm": state["anno_llm"],
        "max_iter": state["max_iter"],
        "anno_strategy": state["anno_strategy"],
        "cell_markers_table": state["cell_markers_table"],
        "exp_summary_dict": state["exp_summary_dict"],
        "metadata_dict": state["metadata_dict"],
        "cluster":s,
        }) for s in state["clusters"]]

max_retry = 3
#Build subgraph 
maingraph_builder = StateGraph(MainGraphState)
#nodes
maingraph_builder.add_node("get_metadata", node_get_metadata, retry=RetryPolicy(max_attempts=max_retry))
maingraph_builder.add_node("get_targetcell", node_get_target_celltypes, retry=RetryPolicy(max_attempts=max_retry))
maingraph_builder.add_node("scrape_markers", node_scrape_markers, retry=RetryPolicy(max_attempts=max_retry))
maingraph_builder.add_node("get_expression", node_get_expresssummary, retry=RetryPolicy(max_attempts=max_retry))
maingraph_builder.add_node("annotation_subgraph", node_annotation_mapped, retry=RetryPolicy(max_attempts=max_retry))
maingraph_builder.add_node("normalize_celltype", node_normalize_celltype, retry=RetryPolicy(max_attempts=max_retry))
#edges
maingraph_builder.add_edge(START, "get_metadata")
maingraph_builder.add_edge("get_metadata", "get_targetcell")
maingraph_builder.add_conditional_edges(
    "get_targetcell",
    decision_scrapemarkers,
    {
        "scrape markers":"scrape_markers",
        "skip scrape":"get_expression",
    }
)
maingraph_builder.add_edge("scrape_markers", "get_expression")
maingraph_builder.add_conditional_edges(
    "get_expression",
    map_annotation,
    [
        "annotation_subgraph",
    ]
)
maingraph_builder.add_edge("annotation_subgraph", "normalize_celltype")
maingraph_builder.add_edge("normalize_celltype", END)
#compile
maingraph = maingraph_builder.compile(
    checkpointer=MemorySaver(), #useing standard sql memory saver
)

In [ ]:
display(Image(maingraph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}
res = maingraph.invoke(
    input = {
        #pars
        "anno_llm": think_llm,
        "max_iter" : 5,
        #input
        "input_adata": adata,
        "cluster_key" : "leiden",
        "maker_df_dir": "",
    },
    config = config,
)

In [ ]:
maingraph.get_state(config=config)

In [ ]:
#test maingraph

meta_cols, meta_sum, meta_dic = node_get_metadata(adata=adata, llm=think_llm)

In [ ]:
target_celltypes = node_get_target_celltypes(think_llm, meta_sum)
target_celltypes

In [ ]:
query = node_scrape_markers(think_llm, meta_sum)
query

In [ ]:
reason, markers = scrape_markers(think_llm,'Epithelial cells',query )

In [ ]:
all_dfs = node_scrape_markers(think_llm, meta_sum, target_celltypes)

In [ ]:
cell_mark_df = pd.concat(all_dfs,ignore_index=True)

In [ ]:
type(cell_mark_df['markers'][0])

In [ ]:
cell_mark_df.to_csv("scraped_cell_markers.csv", encoding="utf-8-sig", index=False)

In [ ]:
cell_marker_df = pd.read_csv("scraped_cell_markers.csv")
cell_marker_df

In [ ]:
markers= []
for _, row in cell_marker_df.iterrows():
    if type(row['markers']) == list:
        markers += row['markers']
    elif type(row['markers']) == str:
        markers += ast.literal_eval(row['markers'])

In [ ]:
cell_markers_table = df2markdownTable(cell_mark_df[['cell_type','markers']])

In [ ]:
exp_sum_dic = node_get_expresssummary(adata)

In [ ]:
subgraph

In [ ]:
len(exp_sum_dic[1])